# 나이브베이즈 분류기를 통한 뉴스그룹 데이터 분류

In [5]:
from sklearn.datasets import fetch_20newsgroups
newsdata = fetch_20newsgroups(subset='train')  # subset = all --> 전체 데이터 다운로드  / subset = test --> 훈련용 데이터 다운로드
newsdata.keys()
# data : 뉴스 기사 본문
# filenames : 파일명
# target_names : 정답값
# target : 0~19 

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [6]:
print(len(newsdata.data),len(newsdata.filenames),len(newsdata.target_names),len(newsdata.target))
# 훈련용 데이터 개수 11314 
# 레이블 개수 : 20개

11314 11314 20 11314


In [7]:
newsdata.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [9]:
print(newsdata.target[0])  # target 값 : 0~19
print(newsdata.target_names[7])  # 0번째 데이터의 정답값 : rec.autos

7
rec.autos


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB   # 다중분류용 나이브 베이즈 모듈
from sklearn.metrics import accuracy_score

In [11]:
dtmvector = CountVectorizer()  # countvectorizer : 각 열 단어들의 출현 빈도수 벡터화
# 모든 단어들을 소문자로 변환시킴
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


* 물론, DTM을 그대로 나이브 베이즈 분류기에 사용할 수도 있겠지만 DTM 행렬 대신 TF-IDF 가중치를 적용한 TF-IDF 행렬을 입력으로 텍스트 분류를 수행하면, 성능의 개선을 얻을 수도 있습니다. (DTM 챕터 참고) 주의할 점은 TF-IDF 행렬이 항상 DTM으로 수행했을 때보다 성능이 뛰어나지는 않습니다. 사이킷런은 TF-IDF를 자동 계산해주는 TfidVectorizer 클래스를 제공하므로 이를 사용해보겠습니다.

In [12]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [14]:
# 모델 학습
mod = MultinomialNB()
mod.fit(tfidfv,newsdata.target)
MultinomialNB(alpha=1.0,class_prior=None,fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
# 테스트 데이터 가져오기
newsdata_test = fetch_20newsgroups(subset='test',shuffle=True)
X_test_dtm = dtmvector.transform(newsdata_test.data)
tfidfv_test = tfidf_transformer.transform(X_test_dtm)

mod.predict(tfidfv_test)  # 테스트 데이터 예측

array([ 7, 11,  0, ...,  9,  3, 15])

In [16]:
accuracy_score(newsdata_test.target,mod.predict(tfidfv_test))

0.7738980350504514